# Yelp Data Association Script (State-Split Version)

This script merges all filtered Yelp datasets and produces **smaller, state-specific DataFrames** for analysis. 
Each dataset's columns are prefixed to indicate their source:

- `review_` → review information  
- `business_` → business information  
- `checkin_` → checkin summary  
- `tip_` → tip summary  
- `user_` → user information  

### Key Steps

1. **Rename columns** in each dataset (except merge keys) to indicate their source.  
2. **Aggregate** checkins (`total_checkins`) and tips (`num_tips`) to avoid row duplication.  
3. **Merge** reviews with business info, checkins, tips, and user info using `business_id` and `user_id`.  
4. **Split by state**: the resulting DataFrame is broken into smaller DataFrames for each `business_state`.  
5. Each state-specific DataFrame is saved separately (CSV), making it easier to work with large datasets.

> **Note:** Column prefixes ensure that the origin of each field is clear. Splitting by state allows analysis without memory issues caused by a single huge DataFrame.

In [34]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [35]:
# Load all filtered datasets
business_df = pd.read_csv("data-restaurants-only/business.csv")
checkin_df = pd.read_csv("data-restaurants-only/checkin.csv")
review_df = pd.read_csv("data-restaurants-only/review.csv")
tip_df = pd.read_csv("data-restaurants-only/tip.csv")
user_df = pd.read_csv("data-restaurants-only/user.csv")

/var/folders/cl/5mfvhcls2nv2h9gy15m04b640000gn/T/ipykernel_37414/2635155766.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  user_df = pd.read_csv("data-restaurants-only/user.csv")


In [36]:
# Prep dataframes for merging:
# 1. Rename columns to include suffix for source dataframe
# 2. and aggregate tip/checkin data into useful format

review_df_renamed = review_df.rename(
    columns={col: f"review_{col}" for col in review_df.columns if col not in ['review_id', 'business_id', 'user_id']}
)

business_df_renamed = business_df.rename(
    columns={col: f"business_{col}" for col in business_df.columns if col != 'business_id'}
)

user_df_renamed = user_df.rename(
    columns={col: f"user_{col}" for col in user_df.columns if col != 'user_id'}
)

# I am aggregating to total_checkins b/c I think that is the only useful feature I could possibly extract
# I don't necessarily think this will be useful but I am not getting rid of any data at this stage
checkin_df['total_checkins'] = checkin_df['date'].str.count(',')
checkin_summary = checkin_df[['business_id', 'total_checkins']].rename(
    columns={'total_checkins': 'checkin_total_checkins'}
)

# Aggregating tip data: count number of tips per business
tips_summary = tip_df.groupby('business_id').agg({
    'user_id': 'count'  # number of tips
}).rename(columns={'user_id': 'tip_num_tips'}).reset_index()

In [37]:
# Merging all of the data into one associated dataframe

# Start by merging reviews with business info
df = review_df_renamed.merge(business_df_renamed, on='business_id', how='left')

# Merge checkin summary data
df = df.merge(checkin_summary, on='business_id', how='left')

# Merge tip summary data
df = df.merge(tips_summary, on='business_id', how='left')

# Finally merge with the user data
df = df.merge(user_df_renamed, on='user_id', how='left')

In [38]:
# Now we have one big associated dataframe ready for modeling!!
df.head()

,review_id,user_id,business_id,review_stars,review_useful,review_funny,review_cool,review_text,review_date,business_name,...,user_compliment_more,user_compliment_profile,user_compliment_cute,user_compliment_list,user_compliment_note,user_compliment_plain,user_compliment_cool,user_compliment_funny,user_compliment_writer,user_compliment_photos
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Turning Point of North Wales,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0.0,0.0,0.0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,Kettle Restaurant,...,14.0,3.0,1.0,1.0,63.0,96.0,86.0,86.0,49.0,27.0
2,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,Zaika,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,Melt,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0
4,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,Dmitri's,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# Saving this huge dataframe to csv leads to a memory bottleneck
# So I am instead going to break it up by state (location)
states = df.business_state.unique()
for state in states:
    df_state = df[df['business_state'] == state].copy()
    filename = f"associated-data-by-state/consolidated_{state}.csv"
    df_state.to_csv(filename, index=False)
    print("Saved: ", filename)

Saved:  associated-data-by-state/consolidated_PA.csv
Saved:  associated-data-by-state/consolidated_AZ.csv
Saved:  associated-data-by-state/consolidated_LA.csv
Saved:  associated-data-by-state/consolidated_CA.csv
Saved:  associated-data-by-state/consolidated_IN.csv
Saved:  associated-data-by-state/consolidated_MO.csv
Saved:  associated-data-by-state/consolidated_FL.csv
Saved:  associated-data-by-state/consolidated_TN.csv
Saved:  associated-data-by-state/consolidated_NV.csv
Saved:  associated-data-by-state/consolidated_NJ.csv
Saved:  associated-data-by-state/consolidated_IL.csv
Saved:  associated-data-by-state/consolidated_ID.csv
Saved:  associated-data-by-state/consolidated_AB.csv
Saved:  associated-data-by-state/consolidated_DE.csv
Saved:  associated-data-by-state/consolidated_HI.csv
Saved:  associated-data-by-state/consolidated_NC.csv
Saved:  associated-data-by-state/consolidated_CO.csv
Saved:  associated-data-by-state/consolidated_MT.csv
Saved:  associated-data-by-state/consolidated_